In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout
import threading
import cvzone

In [23]:
width_img = 640
height_img = 480

stopButton = widgets.ToggleButton(description='Stop', disabled=False)

val1 = widgets.IntRangeSlider(min=0, max=255, step=1, value=[96, 255], layout=Layout(width='80%'), description='Canny_threshold')
val2 = widgets.IntRangeSlider(min=0, max=255, step=1, value=[180, 255], layout=Layout(width='80%'))
val3 = widgets.RadioButtons(options=[True, False], description='L2Gradient', disabled=False)

b1 = [0,255,150,255,0,255]
a = b1.copy()
hue = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[0], a[1]], layout=Layout(width='80%'), description='hue')
sat = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[2], a[3]], layout=Layout(width='80%'), description='sat')
val = widgets.IntRangeSlider(min=0, max=255, step=1, value=[a[4], a[5]], layout=Layout(width='80%'), description='val')

def preprocess(fr, v1, hue, sat, val):
    mask = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    mask = cv2.GaussianBlur(mask, (5,5), 2)
    mask = cv2.Canny(mask, v1.value[0], v1.value[1], apertureSize=5)
	# mask = cv2.dilate(mask, np.ones((5,5)), iterations=2)
    # mask = cv2.erode(mask, np.ones((5,5)), iterations=1)
    return mask

def contours(fr_m, fr):
    biggest = np.array([])
    max_area = 0
    contours, hierarchy = cv2.findContours(fr_m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours, hierarchy = cv2.findContours(fr_m, cv2.RETR_EXTERNAL, method=1)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100_000:
            epsilon = 0.1 * cv2.arcLength(cnt, True)
            # approx = cv2.convexHull(cnt)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            cv2.drawContours(fr, cnt, -1, (0,255,0), 5)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    # cv2.drawContours(fr, biggest, -1, (0,255,0), 30)
    return biggest

def reorder(my_points):
    my_points = my_points.reshape((4,2))
    my_points_new = np.zeros((4,1,2), np.int32)
    add = my_points.sum(1)

    my_points_new[0] = my_points[np.argmin(add)]
    my_points_new[3] = my_points[np.argmax(add)]

    diff = np.diff(my_points, axis=1)

    my_points_new[1] = my_points[np.argmin(diff)]
    my_points_new[2] = my_points[np.argmax(diff)]

    return my_points_new

def get_warp(img, approx):
    biggest = reorder(approx)
    # print(approx.shape)

    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [width_img, 0], [0, height_img], [width_img, height_img]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    img_output = cv2.warpPerspective(img, matrix, (width_img, height_img))

    # img_cropped = img_output[20:img_output.shape[0]-20, 20:img_output.shape[1]-20]
    img_cropped = cv2.resize(img_output, (width_img, height_img))
    return img_cropped

def text_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray)
    for i, region in enumerate(regions):
        x, y, w, h = cv2.boundingRect(region)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

def view(button, val1, hue, sat, val):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)

    while True:
        _, frame = cap.read()
        # frame_height, frame_width, _ = frame.shape
        # print(frame.shape)

        frame_masked = preprocess(frame, val1, hue, sat, val)
        frame_contours = contours(frame_masked, frame)

        if frame_contours.size != 0:
            img_warped = get_warp(frame, frame_contours)
            frame_detect = text_detection(img_warped)
            # print(biggest)
        else:
            img_warped = frame.copy()

        img_stack = cvzone.stackImages([
            frame,
            frame_masked,
            img_warped,
            # frame_hsv,
        ], 3, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

display(stopButton, val1, hue, sat, val)
thread = threading.Thread(target=view, args=(stopButton, val1, hue, sat, val))
thread.start()

ToggleButton(value=False, description='Stop')

IntRangeSlider(value=(96, 255), description='Canny_threshold', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(0, 255), description='hue', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(150, 255), description='sat', layout=Layout(width='80%'), max=255)

IntRangeSlider(value=(0, 255), description='val', layout=Layout(width='80%'), max=255)

None